In [3]:
import findspark

In [4]:
findspark.init() #or put path of spark in the brackets

In [5]:
import pyspark

In [6]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("TelecomDataProcessing").getOrCreate()

# Load the dataset (assuming a CSV file)
df = spark.read.csv(r"D:\Data Science\Pyspark\WA_Fn-UseC_-Telco-Customer-Churn.csv", header=True, inferSchema=True)


In [7]:
df.columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [8]:

# Select relevant columns
df_selected = df.select("gender", 
    "SeniorCitizen", 
    "Partner", 
    "Dependents", 
    "tenure", 
    "PhoneService", 
    "MultipleLines", 
    "InternetService", 
    "OnlineSecurity", 
    "OnlineBackup", 
    "DeviceProtection", 
    "TechSupport", 
    "StreamingTV", 
    "StreamingMovies", 
    "Contract", 
    "PaperlessBilling", 
    "PaymentMethod", 
    "MonthlyCharges", 
    "TotalCharges", 
    "Churn")

# Show the selected columns
df_selected.show(5)


+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|         No|             No|Month-to-month|     

In [9]:
# Filter data where Churn is 'True'
df_selected = df.filter(df["Churn"] == "Yes")

# Show the filtered data
df_selected.show(5)


+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|3668-QPYBK|  Male|            0|     No|        No|     2|         Yes|           No|            DSL|           Yes|         Yes|              No|         No|         No|    

In [10]:
df.dropna()

DataFrame[customerID: string, gender: string, SeniorCitizen: int, Partner: string, Dependents: string, tenure: int, PhoneService: string, MultipleLines: string, InternetService: string, OnlineSecurity: string, OnlineBackup: string, DeviceProtection: string, TechSupport: string, StreamingTV: string, StreamingMovies: string, Contract: string, PaperlessBilling: string, PaymentMethod: string, MonthlyCharges: double, TotalCharges: string, Churn: string]

In [11]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("telecom_data")

# Perform SQL queries
#  query with backticks around the column name with spaces
result = spark.sql("SELECT Churn, AVG(`MonthlyCharges`) as AvgMonthlyCharge FROM telecom_data GROUP BY Churn")
result.show()


+-----+----------------+
|Churn|AvgMonthlyCharge|
+-----+----------------+
|   No|61.2651236953999|
|  Yes|74.4413322632423|
+-----+----------------+



In [12]:
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
from pyspark.sql.functions import col

# Convert 'TotalCharges' to a numeric type
df = df.withColumn("TotalCharges", col("TotalCharges").cast("float"))

# Drop rows with null values in 'MonthlyCharges' or 'TotalCharges'
df = df.dropna(subset=["MonthlyCharges", "TotalCharges"])

# Apply VectorAssembler to create a feature vector including 'tenure'
assembler = VectorAssembler(inputCols=["tenure", "MonthlyCharges", "TotalCharges"], outputCol="features")
df_vector = assembler.transform(df)

# Apply MinMaxScaler for normalization
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(df_vector)
df_normalized = scaler_model.transform(df_vector)

# Show the normalized features
df_normalized.select("scaledFeatures").show(5, truncate=False)



+---------------------------------------------------------------+
|scaledFeatures                                                 |
+---------------------------------------------------------------+
|[0.0,0.11542288557213931,0.001275098245151048]                 |
|[0.4647887323943662,0.3850746268656717,0.2158666100576614]     |
|[0.014084507042253521,0.35422885572139307,0.010310408988541875]|
|[0.619718309859155,0.23930348258706463,0.21024117720577754]    |
|[0.014084507042253521,0.5218905472636816,0.015330025114455744] |
+---------------------------------------------------------------+
only showing top 5 rows



In [13]:
from pyspark.ml.feature import StandardScaler

# Apply StandardScaler for standardization
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withMean=True, withStd=True)
scaler_model = scaler.fit(df_vector)
df_standardized = scaler_model.transform(df_vector)

# Show the standardized features
df_standardized.select("scaledFeatures").show(5)


+--------------------+
|      scaledFeatures|
+--------------------+
|[-1.2801570035428...|
|[0.06429811287810...|
|[-1.2394159394088...|
|[0.51244981835175...|
|[-1.2394159394088...|
+--------------------+
only showing top 5 rows



In [14]:
from pyspark.sql.functions import col
# Now apply StringIndexer
from pyspark.ml.feature import StringIndexer

# Index the categorical column
indexer = StringIndexer(inputCol="Churn", outputCol="ChurnIndex")
df_indexed = indexer.fit(df).transform(df)

# Show the indexed data
df_indexed.select("Churn", "ChurnIndex").show(5)


+-----+----------+
|Churn|ChurnIndex|
+-----+----------+
|   No|       0.0|
|   No|       0.0|
|  Yes|       1.0|
|   No|       0.0|
|  Yes|       1.0|
+-----+----------+
only showing top 5 rows



In [15]:
from pyspark.ml.feature import OneHotEncoder

# Apply OneHotEncoder to the indexed column
encoder = OneHotEncoder(inputCol="ChurnIndex", outputCol="ChurnVec")
df_encoded = encoder.fit(df_indexed).transform(df_indexed)

# Show the encoded data
df_encoded.select("Churn", "ChurnIndex", "ChurnVec").show(5)


+-----+----------+-------------+
|Churn|ChurnIndex|     ChurnVec|
+-----+----------+-------------+
|   No|       0.0|(1,[0],[1.0])|
|   No|       0.0|(1,[0],[1.0])|
|  Yes|       1.0|    (1,[],[])|
|   No|       0.0|(1,[0],[1.0])|
|  Yes|       1.0|    (1,[],[])|
+-----+----------+-------------+
only showing top 5 rows



In [16]:
categorical_cols = ["gender", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", 
                    "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", 
                    "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod"]

In [17]:
indexers = [StringIndexer(inputCol=col, outputCol=col + "_Index") for col in categorical_cols]

In [18]:
for indexer in indexers:
    df_selected = indexer.fit(df_selected).transform(df_selected)

In [19]:
encoder = OneHotEncoder(inputCols=[col + "_Index" for col in categorical_cols],
                        outputCols=[col + "_Vec" for col in categorical_cols])

In [20]:
df_selected = df_selected.withColumn("TotalCharges", col("TotalCharges").cast("float"))

df_encoded = encoder.fit(df_selected).transform(df_selected)

In [25]:
from pyspark.sql.types import IntegerType, DoubleType, FloatType

numeric_types = [IntegerType, DoubleType, FloatType]

# Filter numerical columns by checking the column types and excluding the categorical columns
numerical_cols = [col for col, dtype in df.dtypes if dtype in ['int', 'double', 'float'] and col not in categorical_cols]

In [26]:
assembler = VectorAssembler(inputCols=[col + "_Vec" for col in categorical_cols] + numerical_cols,
                            outputCol="features")

df_vector = assembler.transform(df_encoded)


# question 2

# Consider USA House Pricing  dataset and prepare a dataframe of it.Perform following operations on it.
●	 Apply basic transformation functions on it to get insights on telecom data . 

●	Create a SQL table of it to perform basic SQL transformations on it.

●	Perform preprocessing transformations on relevant features 
5.	Normalization
6.	Standardization
7.	Encoding
8.	Feature engineering



In [27]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("HousingDataProcessing").getOrCreate()

# Load the dataset
df = spark.read.csv(r"D:\Data Science\Pyspark\Housing.csv", header=True, inferSchema=True)

# Inspect the dataset
df.printSchema()
df.show(5)


root
 |-- price: integer (nullable = true)
 |-- area: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- stories: integer (nullable = true)
 |-- mainroad: string (nullable = true)
 |-- guestroom: string (nullable = true)
 |-- basement: string (nullable = true)
 |-- hotwaterheating: string (nullable = true)
 |-- airconditioning: string (nullable = true)
 |-- parking: integer (nullable = true)
 |-- prefarea: string (nullable = true)
 |-- furnishingstatus: string (nullable = true)

+--------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|   price|area|bedrooms|bathrooms|stories|mainroad|guestroom|basement|hotwaterheating|airconditioning|parking|prefarea|furnishingstatus|
+--------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|13300000|7420|       4|        2|      3|

In [28]:

# Print the schema to see the available column names
df.printSchema()




root
 |-- price: integer (nullable = true)
 |-- area: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- stories: integer (nullable = true)
 |-- mainroad: string (nullable = true)
 |-- guestroom: string (nullable = true)
 |-- basement: string (nullable = true)
 |-- hotwaterheating: string (nullable = true)
 |-- airconditioning: string (nullable = true)
 |-- parking: integer (nullable = true)
 |-- prefarea: string (nullable = true)
 |-- furnishingstatus: string (nullable = true)



In [29]:
# Select relevant columns based on the correct column names
df_selected = df.select("price", "bedrooms", "bathrooms", "area", "stories", "parking", "mainroad", "guestroom")

# Show the selected columns
df_selected.show(5)


+--------+--------+---------+----+-------+-------+--------+---------+
|   price|bedrooms|bathrooms|area|stories|parking|mainroad|guestroom|
+--------+--------+---------+----+-------+-------+--------+---------+
|13300000|       4|        2|7420|      3|      2|     yes|       no|
|12250000|       4|        4|8960|      4|      3|     yes|       no|
|12250000|       3|        2|9960|      2|      2|     yes|       no|
|12215000|       4|        2|7500|      2|      3|     yes|       no|
|11410000|       4|        1|7420|      2|      2|     yes|      yes|
+--------+--------+---------+----+-------+-------+--------+---------+
only showing top 5 rows



In [30]:
# Filter houses priced above $500,000
df_filtered = df.filter(df["Price"] > 500000)

# Show the filtered data
df_filtered.show(5)


+--------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|   price|area|bedrooms|bathrooms|stories|mainroad|guestroom|basement|hotwaterheating|airconditioning|parking|prefarea|furnishingstatus|
+--------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|13300000|7420|       4|        2|      3|     yes|       no|      no|             no|            yes|      2|     yes|       furnished|
|12250000|8960|       4|        4|      4|     yes|       no|      no|             no|            yes|      3|      no|       furnished|
|12250000|9960|       3|        2|      2|     yes|       no|     yes|             no|             no|      2|     yes|  semi-furnished|
|12215000|7500|       4|        2|      2|     yes|       no|     yes|             no|            yes|      3|     yes|       furnished|
|11410000|7420|       4|        1|      2

In [31]:
# Group by 'Bedrooms' and calculate the average price
df_grouped = df.groupBy("Bedrooms").agg({"Price": "mean"}).withColumnRenamed("avg(Price)", "AvgPrice")

# Show the grouped and aggregated results
df_grouped.show()


+--------+------------------+
|Bedrooms|          AvgPrice|
+--------+------------------+
|       1|         2712500.0|
|       6|         4791500.0|
|       3| 4954598.133333334|
|       5|         5819800.0|
|       4| 5729757.894736842|
|       2|3632022.0588235296|
+--------+------------------+



In [32]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("housing_data")

# Perform SQL queries
result = spark.sql("SELECT Bedrooms, AVG(Price) as AvgPrice FROM housing_data GROUP BY Bedrooms")
result.show()


+--------+------------------+
|Bedrooms|          AvgPrice|
+--------+------------------+
|       1|         2712500.0|
|       6|         4791500.0|
|       3| 4954598.133333334|
|       5|         5819800.0|
|       4| 5729757.894736842|
|       2|3632022.0588235296|
+--------+------------------+



In [33]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler

# Replace with the correct column names
assembler = VectorAssembler(inputCols=["area", "parking"], outputCol="features")

# Transform the data
df_vector = assembler.transform(df)

# Apply MinMaxScaler for normalization
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
df_scaled = scaler.fit(df_vector).transform(df_vector)

# Show the resulting DataFrame
df_scaled.select("features", "scaledFeatures").show(5)



+------------+--------------------+
|    features|      scaledFeatures|
+------------+--------------------+
|[7420.0,2.0]|[0.39656357388316...|
|[8960.0,3.0]|[0.50240549828178...|
|[9960.0,2.0]|[0.57113402061855...|
|[7500.0,3.0]|[0.40206185567010...|
|[7420.0,2.0]|[0.39656357388316...|
+------------+--------------------+
only showing top 5 rows



In [34]:
from pyspark.ml.feature import StandardScaler

# Apply StandardScaler for standardization
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withMean=True, withStd=True)
scaler_model = scaler.fit(df_vector)
df_standardized = scaler_model.transform(df_vector)

# Show the standardized features
df_standardized.select("scaledFeatures").show(5)


+--------------------+
|      scaledFeatures|
+--------------------+
|[1.04576554797932...|
|[1.75539685028965...|
|[2.21619639724442...|
|[1.08262951173570...|
|[1.04576554797932...|
+--------------------+
only showing top 5 rows



In [35]:
from pyspark.ml.feature import StringIndexer

# Apply StringIndexer for encoding categorical columns
indexer = StringIndexer(inputCols=["mainroad", "guestroom"], outputCols=["mainroad_indexed", "guestroom_indexed"])
indexed_df = indexer.fit(df).transform(df)

indexed_df.show(5)


+--------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+----------------+-----------------+
|   price|area|bedrooms|bathrooms|stories|mainroad|guestroom|basement|hotwaterheating|airconditioning|parking|prefarea|furnishingstatus|mainroad_indexed|guestroom_indexed|
+--------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+----------------+-----------------+
|13300000|7420|       4|        2|      3|     yes|       no|      no|             no|            yes|      2|     yes|       furnished|             0.0|              0.0|
|12250000|8960|       4|        4|      4|     yes|       no|      no|             no|            yes|      3|      no|       furnished|             0.0|              0.0|
|12250000|9960|       3|        2|      2|     yes|       no|     yes|             no|             no|      2|     yes|  semi-furnished|    

In [36]:
from pyspark.sql.functions import when

# Example: Create a new feature based on the area
df = df.withColumn("large_area", when(df.area > 5000, 1).otherwise(0))

df.select("area", "large_area").show(5)


+----+----------+
|area|large_area|
+----+----------+
|7420|         1|
|8960|         1|
|9960|         1|
|7500|         1|
|7420|         1|
+----+----------+
only showing top 5 rows

